$\newcommand{\To}{\Rightarrow}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import BoolType, NatType
from kernel.term import Term, Var, And, Or, Implies
from kernel.thm import Thm
from logic import basic
from syntax.settings import settings

basic.load_theory('nat')

## Sequents

In this section, we start the discussion of theorems and proofs. We follow a method of describing proofs known as *natural deduction*. In natural deduction, each step of proof results in a *sequent*, consisting of a set of assumptions (called *antecedents*) and a conclusion (called *consequent*). Each antecedent and the consequent must be a term of type boolean. A sequent with antecedent $A_1, \dots, A_n$ and consequent $C$ is written as:

$$ A_1, \dots, A_n \vdash C $$

Note even though the antecedents are considered as a set, we will often write them as an ordered tuple for convenience. It is implicitly understood that antecedents are sets when taking unions and when subtracting an element.

In Python, a theorem is constructed from antecedents and consequent as follows:

In [3]:
A = Var("A", BoolType)
B = Var("B", BoolType)
C = Var("C", BoolType)
th = Thm([A, B], C)
print(th)

A, B |- C


Note the antecedents and the consequent are separated by the `|-` sign. If unicode is on, the symbol $\vdash$ is displayed:

In [4]:
settings.unicode = True
print(th)

A, B ⊢ C


When theorems are constructed, it is not immediately checked that the antecedents and the consequent have the right type (boolean). We can check it using the `check_thm_type` method:

In [5]:
th.check_thm_type()  # Passes

x = Var("x", NatType)
th2 = Thm([A, x], C)
th2.check_thm_type()  # raises TypeCheckException

TypeCheckException: expect boolean type for propositions

The antecedents and consequent of a theorem is accessed using `hyps` (for hypothesis) and `prop` (for proposition).

In [6]:
print('hyps:', ', '.join(str(hyp) for hyp in th.hyps))
print('prop:', th.prop)

hyps: A, B
prop: C


## Primitive deduction rules

A *deduction system* is defined by a set of rules that are assumed to always yield valid sequents. We call these the *primitive deduction rules*. They form the core of the logical theorem. For higher-order logic, the set of primitive deduction rules is mostly standardized, although there are some variations among the implementations. We introduce our set of rules starting in this section.

### Assumption

The first rule is the *assume* rule. It states that given any (boolean) term $A$, the sequent $A \vdash A$ holds. It can be written graphically as follows:

$$ \frac{}{A \vdash A} \hbox{ assume} $$

In this way of displaying rules, the output of the rule written below the horizontal line, and the inputs are written above the line. Since the assume rule does not have any inputs, the area above the line is empty. All variables appearing in the rule are arbitrary: they can be substituted for any term of the right type.

The assume rule can be invoked as follows:

In [7]:
print(Thm.assume(A))

A ⊢ A


Of course, the input does not have to be $A$, or a variable, but any term of boolean type. For example:

In [8]:
x = Var("x", NatType)
y = Var("y", NatType)
t = (x < y + 2)
print(Thm.assume(t))

x < y + 2 ⊢ x < y + 2


### Implication

Next, we discuss two rules for implication. Recall that implication is formed using `Implies`:

In [9]:
A = Var('A', BoolType)
B = Var('B', BoolType)
print(Implies(A, B))

A ⟶ B


The introduction rule for implication states how to obtain a theorem whose consequent is an implication. It is written graphically as follows:

$$ \frac{A \vdash B}{\vdash A \to B} \hbox{ implies_intr} $$

The corresponding Python function is `implies_intr`, which takes two arguments: the assumption of the implication $A$, and the input theorem $\mathit{th}$. It removes $A$ from the antecedents of $\mathit{th}$, and changes the consequent of $\mathit{th}$ to $A \to C$, where $C$ is the original consequent.

In [10]:
th = Thm([A], B)
print("th: ", th)

th2 = Thm.implies_intr(A, th)
print("th2:", th2)

th:  A ⊢ B
th2: ⊢ A ⟶ B


The input theorem may contain multiple antecedents:

In [11]:
th = Thm([A, B], C)
print("th: ", th)

th2 = Thm.implies_intr(A, th)
print("th2:", th2)

th3 = Thm.implies_intr(B, th)
print("th3:", th3)

th:  A, B ⊢ C
th2: B ⊢ A ⟶ C
th3: A ⊢ B ⟶ C


The input theorem may also have antecedents that do not contain $A$. In this case, the set of antecedents is unchanged:

In [12]:
th = Thm([B], C)
print("th: ", th)

th2 = Thm.implies_intr(A, th)
print("th2:", th2)

th:  B ⊢ C
th2: B ⊢ A ⟶ C


The elimination rule for implication can be considered as the dual of the introduction rule. It states how to make use of a theorem whose consequent is an implication:

$$ \frac{\vdash A \to B \quad \vdash A}{\vdash B} \hbox{ implies_elim} $$

It is implemented as the function `implies_elim`:

In [13]:
th1 = Thm([], Implies(A, B))
th2 = Thm([], A)
th3 = Thm.implies_elim(th1, th2)
print("th1:", th1)
print("th2:", th2)
print("th3:", th3)

th1: ⊢ A ⟶ B
th2: ⊢ A
th3: ⊢ B


Each of the two arguments of `implies_elim` can have additional antecedents. They are combined in the results:

In [14]:
D = Var("D", BoolType)
th1 = Thm([C], Implies(A, B))
th2 = Thm([D], A)
th3 = Thm.implies_elim(th1, th2)
print("th1:", th1)
print("th2:", th2)
print("th3:", th3)

th1: C ⊢ A ⟶ B
th2: D ⊢ A
th3: C, D ⊢ B


The function `implies_elim` checks that its inputs are valid. That is, the first argument is an implication, and the assumption of the implication agrees with the second argument. If the arguments are invalid for any reason, it raises an `InvalidDerivationException`:

In [15]:
th1 = Thm([], Implies(A, B))
th2 = Thm([], C)
th3 = Thm.implies_elim(th1, th2)  # raises InvalidDerivationException

InvalidDerivationException: implies_elim: A != C

### Proofs with implication

A *proof* in higher-order logic is a sequence of steps, where each step consists of applying a primitive deduction rule. At each step, the inputs to the deduction rule (if any) must come from sequents obtained in earlier steps. The result of the proof is the sequent obtained by the last step.

We now show some examples of proofs concerning implications, making use of the three primitive deduction rules shown above.

#### Example:

Prove $\vdash A \to A$.

#### Solution:

First use assume to obtain $A \vdash A$, then use introduction of implication to get $\vdash A \to A$. This can be written formally as follows:

0. $A \vdash A$ by assume A.
1. $\vdash A \to A$ by implies_intr A from 0.

In the above format for proofs, each line is a step of deduction. The lines are numbered starting from zero. Each line begins by the sequent obtained by that line, followed by the deduction rule used, arguments of the rule, and input sequents of the rule (referred to by line numbers). This format will be used for describing proofs both in text as well as in Python code.

We can check the above deduction using the following code:

In [16]:
th0 = Thm.assume(A)
th1 = Thm.implies_intr(A, th0)
print(th1)

⊢ A ⟶ A


#### Example:

Prove $\vdash A \to (A \to B) \to B$.

#### Solution:

0. $A \to B \vdash A \to B$ by assume $A \to B$.
1. $A \vdash A$ by assume $A$.
2. $A \to B, A \vdash B$ by implies_elim from 0, 1.
3. $A \vdash (A \to B) \to B$ by implies_intr $A \to B$ from 2.
4. $\vdash A \to (A \to B) \to B$ by implies_intr $A$ from 3.

The Python code is as follows:

In [17]:
th0 = Thm.assume(Implies(A, B))
th1 = Thm.assume(A)
th2 = Thm.implies_elim(th0, th1)
th3 = Thm.implies_intr(Implies(A, B), th2)
th4 = Thm.implies_intr(A, th3)
print(th4)

⊢ A ⟶ (A ⟶ B) ⟶ B


#### Example:

Prove $\vdash (A \to B) \to (B \to C) \to A \to C$ (syllogism).

#### Solution:

0. $A \to B \vdash A \to B$ by assume $A \to B$.
1. $B \to C \vdash B \to C$ by assume $B \to C$.
2. $A \vdash A$ by assume $A$.
3. $A \to B, A \vdash B$ by implies_elim from 0, 2.
4. $A \to B, B \to C, A \vdash C$ by implies_elim from 1, 3.
5. $A \to B, B \to C \vdash A \to C$ by implies_intr $A$ from 4.
6. $A \to B \vdash (B \to C) \to A \to C$ by implies_intr $B \to C$ from 5.
7. $\vdash (A \to B) \to (B \to C) \to A \to C$ by implies_intr $A \to B$ from 6.

In Python code:

In [18]:
th0 = Thm.assume(Implies(A, B))
th1 = Thm.assume(Implies(B, C))
th2 = Thm.assume(A)
th3 = Thm.implies_elim(th0, th2)
th4 = Thm.implies_elim(th1, th3)
th5 = Thm.implies_intr(A, th4)
th6 = Thm.implies_intr(Implies(B, C), th5)
th7 = Thm.implies_intr(Implies(A, B), th6)
print(th7)

⊢ (A ⟶ B) ⟶ (B ⟶ C) ⟶ A ⟶ C


With these examples, we gained some experience working with the first three primitive deduction rules. In the next section, we introduce rules for working with equality.